In [6]:
import sys
import os

# 将项目根目录添加到 Python 路径
sys.path.append(os.path.abspath(os.path.join(os.path.dirname('Time-Series-Library-main_Cursor'), 'D:\\MR\\Desktop\\Time-Series-Library-main\\Time-Series-Library-main_Cursor')))

In [7]:
from models.TimesNet import Model
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from models.TimesNet import Model                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [8]:
# 设置随机种子以确保结果可重复
torch.manual_seed(42)
np.random.seed(42)

# 读取数据
data = pd.read_excel('数据.xlsx')

# 将销售日期列转换为datetime类型
data['销售日期'] = pd.to_datetime(data['销售日期'])

# 提取需要的列
columns = ['销量(千克)', '销售单价(元/千克)', '批发价格(元/千克)']

In [31]:
def create_sequences(data, seq_length):
    sequences = []
    targets = []
    for i in range(len(data) - seq_length):
        seq = data[i:i+seq_length]
        target = data[i+seq_length]
        sequences.append(seq)
        targets.append(target)
    return np.array(sequences), np.array(targets).reshape(-1, 1)

def create_time_features(dates):
    return np.column_stack([
        dates.dt.dayofweek,
        dates.dt.dayofyear,
        dates.dt.month,
        dates.dt.is_month_start.astype(int),
        dates.dt.is_month_end.astype(int)
    ]).astype(float)

def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs):
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        for batch in train_loader:
            optimizer.zero_grad()
            x_enc, x_mark_enc, x_dec, x_mark_dec, y = batch
            output = model(x_enc, x_mark_enc, x_dec, x_mark_dec)
            loss = criterion(output, y)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for batch in val_loader:
                x_enc, x_mark_enc, x_dec, x_mark_dec, y = batch
                output = model(x_enc, x_mark_enc, x_dec, x_mark_dec)
                loss = criterion(output, y)
                val_loss += loss.item()
        
        print(f'Epoch {epoch+1}, Train Loss: {train_loss/len(train_loader):.4f}, Val Loss: {val_loss/len(val_loader):.4f}')

def evaluate_model(model, test_loader, scaler):
    model.eval()
    predictions = []
    actuals = []
    with torch.no_grad():
        for batch in test_loader:
            x_enc, x_mark_enc, x_dec, x_mark_dec, y = batch
            output = model(x_enc, x_mark_enc, x_dec, x_mark_dec)
            predictions.extend(scaler.inverse_transform(output.cpu().numpy()).flatten())
            actuals.extend(scaler.inverse_transform(y.cpu().numpy()).flatten())
    return np.array(predictions), np.array(actuals)

def plot_results(predictions, actuals, title):
    plt.figure(figsize=(12, 6))
    plt.plot(actuals, label='实际值')
    plt.plot(predictions, label='预测值')
    plt.title(title)
    plt.legend()
    plt.show()

In [13]:
# 模型参数
seq_length = 30
pred_length = 7
batch_size = 32
num_epochs = 100

In [14]:
class Config:
    task_name = 'long_term_forecast'
    seq_len = 30
    label_len = 1
    pred_len = 1
    top_k = 5
    num_kernels = 6
    d_model = 64
    d_ff = 128
    e_layers = 2
    embed = 'timeF'
    freq = 'd'
    dropout = 0.05
    enc_in = 1  # 输入特征维度
    dec_in = 1  # 解码器输入维度
    c_out = 1  # 输出维度
    d_layers = 1
    n_heads = 8
    activation = 'gelu'
    output_attention = False
    distil = True

for column in columns:
    print(f"\n处理 {column}")
    
    # 准备数据
    data_series = data[column].values.reshape(-1, 1)
    scaler = MinMaxScaler()
    data_scaled = scaler.fit_transform(data_series)
    
    # 创建序列
    X, y = create_sequences(data_scaled, seq_length)
    
    # 分割数据
    X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=30, shuffle=False)
    X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.2, shuffle=False)
    
    # 准备时间特征
    time_features = (data['销售日期'].dt.dayofyear.values / 365.0).reshape(-1, 1)
    time_features_scaled = MinMaxScaler().fit_transform(time_features)
    
    # 准备数据加载器
    train_dataset = TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(time_features_scaled[:X_train.shape[0]]),
                                  torch.FloatTensor(X_train[:, -pred_length:]), torch.FloatTensor(time_features_scaled[:X_train.shape[0]]),
                                  torch.FloatTensor(y_train))
    val_dataset = TensorDataset(torch.FloatTensor(X_val), torch.FloatTensor(time_features_scaled[X_train.shape[0]:X_train.shape[0]+X_val.shape[0]]),
                                torch.FloatTensor(X_val[:, -pred_length:]), torch.FloatTensor(time_features_scaled[X_train.shape[0]:X_train.shape[0]+X_val.shape[0]]),
                                torch.FloatTensor(y_val))
    test_dataset = TensorDataset(torch.FloatTensor(X_test), torch.FloatTensor(time_features_scaled[-X_test.shape[0]:]),
                                 torch.FloatTensor(X_test[:, -pred_length:]), torch.FloatTensor(time_features_scaled[-X_test.shape[0]:]),
                                 torch.FloatTensor(y_test))
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)
    test_loader = DataLoader(test_dataset, batch_size=batch_size)
    
    # 定义模型
    class Config:
        task_name = 'long_term_forecast'
        seq_len = seq_length
        label_len = pred_length
        pred_len = 1
        top_k = 5
        num_kernels = 6
        d_model = 64
        d_ff = 128
        e_layers = 2
        embed = 'timeF'
        freq = 'd'
        dropout = 0.05
        enc_in = 1
        c_out = 1
    
    config = Config()
    model = Model(config)
    
    # 训练模型
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs)
    
    # 评估模型
    predictions, actuals = evaluate_model(model, test_loader, scaler)
    
    # 可视化结果
    plot_results(predictions, actuals, f'{column} 预测')


处理 销量(千克)


RuntimeError: mat1 and mat2 shapes cannot be multiplied (32x1 and 3x64)

In [33]:
# 在主循环中，对每个列进行处理
for column in columns:
    print(f"处理 {column}")
    
    # 准备数据
    series = data[column].values.reshape(-1, 1)
    scaler = MinMaxScaler()
    series_scaled = scaler.fit_transform(series)
    
    # 创建序列
    seq_length = 30
    pred_length = 1
    X, y = create_sequences(series_scaled, seq_length)
    
    # 准备时间特征
    time_features = create_time_features(data['销售日期'])
    time_features_scaled = MinMaxScaler().fit_transform(time_features)
    
    # 确保时间特征的长度与X匹配
    time_features_scaled = time_features_scaled[seq_length:len(X)+seq_length]
    
    # 分割测试集
    test_size = 30
    X_train_val, X_test = X[:-test_size], X[-test_size:]
    y_train_val, y_test = y[:-test_size], y[-test_size:]
    time_features_train_val, time_features_test = time_features_scaled[:-test_size], time_features_scaled[-test_size:]
    
    # 进一步分割训练集和验证集
    X_train, X_val, y_train, y_val, time_features_train, time_features_val = train_test_split(
        X_train_val, y_train_val, time_features_train_val, test_size=0.2, random_state=42
    )
    
    # 修改时间特征
    time_features = np.column_stack([
        time_features_scaled[:, 0],  # dayofweek
        time_features_scaled[:, 2],  # month
        time_features_scaled[:, 3]   # is_month_start
    ])
    
    # 准备数据加载器
    train_dataset = TensorDataset(
        torch.FloatTensor(X_train),
        torch.FloatTensor(time_features_train),
        torch.FloatTensor(X_train[:, -pred_length:]),
        torch.FloatTensor(time_features_train),
        torch.FloatTensor(y_train)
    )
    val_dataset = TensorDataset(
        torch.FloatTensor(X_val),
        torch.FloatTensor(time_features_val),
        torch.FloatTensor(X_val[:, -pred_length:]),
        torch.FloatTensor(time_features_val),
        torch.FloatTensor(y_val)
    )
    test_dataset = TensorDataset(
        torch.FloatTensor(X_test),
        torch.FloatTensor(time_features_test),
        torch.FloatTensor(X_test[:, -pred_length:]),
        torch.FloatTensor(time_features_test),
        torch.FloatTensor(y_test)
    )
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)
    test_loader = DataLoader(test_dataset, batch_size=batch_size)
    
    # 定义模型
    config = Config()
    model = Model(config)
    
    # 训练模型
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    num_epochs = 50
    
    train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs)
    
    # 评估模型
    predictions, actuals = evaluate_model(model, test_loader, scaler)
   
    
    # 计算评价指标
    mse = np.mean((predictions - actuals)**2)
    rmse = np.sqrt(mse)
    mae = np.mean(np.abs(predictions - actuals))
    
    print(f"{column} 预测结果:")
    print(f"MSE: {mse:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"MAE: {mae:.4f}")
    
    # 可视化结果
    plot_results(predictions, actuals, f'{column} 预测')

处理 销量(千克)


RuntimeError: mat1 and mat2 shapes cannot be multiplied (32x5 and 3x64)